<a href="https://colab.research.google.com/github/damianwgriggs/Perceptual-Grid-Engine-Quantum-Experiment/blob/main/Echo-Stabilized%20Recursive%20Link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- CELL 1: SETUP & SIMULATION ---

# 1. Install Qiskit and IBM Runtime
# We use -q to keep the output clean and minimal
!pip install -q qiskit qiskit-ibm-runtime qiskit-aer numpy

import numpy as np
import socket
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator

print("[Status] Libraries loaded. System Ready.")

# 2. Define the Invention: "Echo-Stabilized Recursive Link"
def create_stabilized_circuit(delay_us=100):
    """
    Constructs the circuit with Active Stabilization (Hahn Echo)
    and Recursive Routing (Mid-Circuit Reset).
    """
    # PHYSICAL REGISTERS
    # q0: Source (Recycled) | q1: Bridge (Recycled) | q2: Buffer (Storage)
    qr = QuantumRegister(3, 'q')

    # CLASSICAL REGISTERS (For Teleportation & Verification)
    cr_hop1 = ClassicalRegister(2, 'hop1') # Outbound
    cr_hop2 = ClassicalRegister(2, 'hop2') # Inbound
    cr_final = ClassicalRegister(1, 'result') # Verification

    qc = QuantumCircuit(qr, cr_hop1, cr_hop2, cr_final)

    # --- PHASE A: INITIALIZATION (Pilot State ~75% |0>) ---
    qc.ry(2 * np.arccos(np.sqrt(0.75)), 0)
    qc.barrier()

    # --- PHASE B: OUTBOUND (q0 -> q2) ---
    qc.h(1)
    qc.cx(1, 2)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, cr_hop1[0])
    qc.measure(1, cr_hop1[1])

    # Feed Forward Correction
    with qc.if_test((cr_hop1[1], 1)):
        qc.x(2)
    with qc.if_test((cr_hop1[0], 1)):
        qc.z(2)
    qc.barrier()

    # --- PHASE C: PARALLEL OPS (The Innovation) ---
    # 1. Recycle Sender & Bridge
    qc.reset(0)
    qc.reset(1)

    # 2. Hahn Echo on Buffer (Stabilization)
    if delay_us > 0:
        half_wait = delay_us / 2
        qc.delay(half_wait, 2, unit='us')
        qc.x(2) # Echo Pulse
        qc.delay(half_wait, 2, unit='us')
        qc.x(2) # Restore

    qc.barrier()

    # --- PHASE D: INBOUND (q2 -> q0) ---
    qc.h(1)
    qc.cx(1, 0)
    qc.cx(2, 1)
    qc.h(2)
    qc.measure(2, cr_hop2[0])
    qc.measure(1, cr_hop2[1])

    with qc.if_test((cr_hop2[1], 1)):
        qc.x(0)
    with qc.if_test((cr_hop2[0], 1)):
        qc.z(0)

    # --- PHASE E: VERIFY ---
    qc.measure(0, cr_final)
    return qc

# 3. Run Logic Simulation
print("[Status] Running Logic Simulation...")
qc_sim = create_stabilized_circuit(delay_us=100)
backend_sim = AerSimulator()

# Transpile & Execute
pm = generate_preset_pass_manager(backend=backend_sim, optimization_level=1)
isa_qc = pm.run(qc_sim)
result = backend_sim.run(isa_qc, shots=4000).result()
counts = result.get_counts()

# 4. Validate
zeros = 0
total = 0
for key, count in counts.items():
    # Key format is "result hop2 hop1", we want the first bit
    if key.split()[0] == '0':
        zeros += count
    total += count

fidelity = (zeros / total) * 100
print(f"\n--- SIMULATION REPORT ---")
print(f"Target: ~75.0%")
print(f"Actual:  {fidelity:.2f}%")

if 73.0 < fidelity < 77.0:
    print("✅ LOGIC VALIDATED: Ready for Hardware.")
else:
    print("⚠️ WARNING: Logic variance detected.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.7 MB/s eta 0:00:00
[Status] Libraries loaded. System Ready.
[Status] Running Logic Simulation...

--- SIMULATION REPORT ---
Target: ~75.0%
Actual:  73.83%
✅ LOGIC VALIDATED: Ready for Hardware.


In [7]:
# --- CELL 2: FINAL INTEGRATION TEST (MICRO-ECHO) ---
import socket
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# 1. AUTHENTICATION
MY_TOKEN = "MYAPIKEYWASHERE"

if MY_TOKEN == "PASTE_YOUR_IBM_KEY_HERE" or MY_TOKEN == "":
    print("STOP: Please paste your IBM Token.")
else:
    try:
        # Check internet
        socket.gethostbyname("cloud.ibm.com")

        print("[Step 1] Logging in...")
        service = QiskitRuntimeService(channel="ibm_quantum_platform", token=MY_TOKEN)

        # Select Hardware
        print("[Step 2] Selecting Processor...")
        backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
        print(f"         Target: {backend.name}")

        # 2. GENERATE COMBINED CIRCUIT
        # We set delay to 20us. This is the key.
        # It is > 0, so the Hahn Echo (Invention A) turns ON.
        # It is short enough that the signal should survive.
        print("[Step 3] Generating 'Micro-Echo' Circuit (20us delay)...")
        qc_final = create_stabilized_circuit(delay_us=20)

        # Transpile
        print("         Transpiling (Opt Level 3)...")
        pm_real = generate_preset_pass_manager(backend=backend, optimization_level=3)
        isa_qc_final = pm_real.run(qc_final)

        # 3. EXECUTE
        print("[Step 4] Submitting Combined Invention...")
        sampler = Sampler(backend)
        job = sampler.run([isa_qc_final])
        print(f"         Job ID: {job.job_id()}")

        # BLOCKING CALL - Wait for results
        result = job.result()

        # 4. ANALYSIS
        print("\n[Step 5] Parsing Data...")
        data_pub = result[0].data
        counts = data_pub.result.get_counts()

        zeros = counts.get('0', 0)
        total_shots = sum(counts.values())
        fidelity = (zeros / total_shots) * 100

        # 5. FINAL REPORT
        print("\n" + "="*40)
        print(f" DAMIAN GRIGGS: COMBINED VALIDATION")
        print("="*40)
        print(f"Echo Status:          ACTIVE (20us Sequence)")
        print(f"Routing Status:       ACTIVE (Recursive Reset)")
        print("-" * 40)
        print(f"Target (Ideal):       75.0%")
        print(f"ACTUAL HARDWARE:      {fidelity:.2f}%")
        print("-" * 40)

        if fidelity > 60.0:
            print("✅ SUCCESS: Both inventions worked simultaneously.")
        else:
            print("❌ DECOHERENCE: The Echo introduced too much noise.")

    except Exception as e:
        print(f"\n[ERROR]: {e}")

qiskit_runtime_service._discover_account:WARNING:2025-12-14 21:17:07,520: Loading account with the given token. A saved account will not be used.


[Step 1] Logging in...


qiskit_runtime_service.__init__:WARNING:2025-12-14 21:17:09,831: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


[Step 2] Selecting Processor...


qiskit_runtime_service.backends:WARNING:2025-12-14 21:17:10,197: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-14 21:17:11,260: Using instance: open-instance, plan: open


         Target: ibm_torino
[Step 3] Generating 'Micro-Echo' Circuit (20us delay)...
         Transpiling (Opt Level 3)...
[Step 4] Submitting Combined Invention...
         Job ID: d4viiluaec6c738se270

[Step 5] Parsing Data...

 DAMIAN GRIGGS: COMBINED VALIDATION
Echo Status:          ACTIVE (20us Sequence)
Routing Status:       ACTIVE (Recursive Reset)
----------------------------------------
Target (Ideal):       75.0%
ACTUAL HARDWARE:      68.70%
----------------------------------------
✅ SUCCESS: Both inventions worked simultaneously.
